In [ ]:
from os import listdir
from os.path import isfile, join

import numpy as np
import cv2
import matplotlib.pyplot as plt

from segmentation_models import PSPNet
from segmentation_models import FPN
from segmentation_models import Unet
from segmentation_models.segmentation_models.backbones import get_preprocessing

from keras import backend as K
import keras
import h5py
import spacexyz
%load_ext autoreload
%autoreload 2
%matplotlib inline
K.tensorflow_backend._get_available_gpus()

In [ ]:
filename = '/scratch2/peilun/SpaceXYZ/oxygen_plans_new.h5'
f = h5py.File(filename, 'r')

# List all groups
print("Keys: %s" % f.keys())
a_group_key = list(f.keys())[0]

# Get the data
data = list(f[a_group_key])
d = np.array(data)
d.shape

# img = d[23,:,:,2]
# plt.imshow(img)
d = d.astype(np.float32)

In [ ]:
# d= plt.imread('wall.png')

In [ ]:
image = d[2,:,:,:]

In [ ]:
image.shape

In [ ]:
fg = np.sum(image[:,:,2:], axis=2)

In [ ]:
fg_copy = np.zeros_like(fg)
fg_copy[:] = fg
fg_copy = (fg_copy*255).astype(np.uint8)

fg_copy[fg < 0.9] = 0

print(fg_copy.min())
plt.imshow(fg_copy, cmap='gray')

In [ ]:
# print(image.shape)
# wall = (image[:,:,1]*255).astype(np.uint8)

# image, contours, hierarchy = cv2.findContours(wall, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
# for contour in contours:
#     area = cv2.contourArea(contour)

#     cv2.drawContours(wall, contour, -1, (0, 255, 0), 3)
# cv2.imshow("window title", wall)


In [ ]:
# countour = np.zeros_like(wall)
# wall[cnts] = 1

In [ ]:
# binarize
# ret, thresh = cv2.threshold(fg_copy,0,255,cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)
ret, thresh = cv2.threshold(fg_copy,0,255,cv2.THRESH_OTSU)

# noise removal
kernel = np.ones((3,3),np.uint8)
opening = cv2.morphologyEx(thresh,cv2.MORPH_OPEN,kernel, iterations = 2)

# sure background area
sure_bg = cv2.dilate(opening,kernel,iterations=3)

# Finding sure foreground area
dist_transform = cv2.distanceTransform(opening,cv2.DIST_L2,5)
ret, sure_fg = cv2.threshold(dist_transform,0.15*dist_transform.max(),255,0)

# Finding unknown region
sure_fg = np.uint8(sure_fg)
unknown = cv2.subtract(sure_bg,sure_fg)

In [ ]:
plt.imshow(unknown)

In [ ]:
# Marker labelling
ret, markers = cv2.connectedComponents(sure_fg)

In [ ]:
# Add one to all labels so that sure background is not 0, but 1
markers = markers+1

In [ ]:
# p# Now, mark the region of unknown with zero
markers[unknown==255] = 0

In [ ]:
plt.imshow(markers)

In [ ]:
img_ = np.tile(image[:,:,4:5], [1, 1, 3])
img_ = (img_*255).astype(np.uint8)
markers = cv2.watershed(img_,markers)
img_[markers == -1] = [255,0,0]

In [ ]:
plt.imshow(markers)

In [ ]:
plt.imsave('seg.png', markers,)

In [ ]:
img_t = np.empty_like (img_)
img_t[:] = img_
img_ = (img_t*255).astype(np.uint8)
img_[img_<200] = 0
plt.imshow(img_)
print(img_.shape)
im = np.tile(img_, (1, 1, 3))
print(im.shape)

In [ ]:
print(img_.shape)
dist_transform = cv2.distanceTransform(img_,cv2.DIST_L2,5)

ret, sure_fg = cv2.threshold(dist_transform,0.2*dist_transform.max(),255,0)
plt.imshow(sure_fg)
sure_fg = sure_fg.astype(np.uint8)

In [ ]:
ret, markers = cv2.connectedComponents(sure_fg)

In [ ]:
markers = markers+1

In [ ]:
print(d.shape)

In [ ]:
markers = cv2.watershed(d[:,:,:2].astype(np.float32),markers)
# img[markers == -1] = [255,0,0]

In [ ]:
# finding fg
dist_transform = cv2.distanceTransform(,cv2.DIST_L2,5)

In [ ]:

ret, thresh = cv2.threshold(img_,0,255,cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)


img_[img_t<thresh] = 0
img_[img_t>=thresh] = 255
plt.imshow(img_)

# img_t = (img_t*255).astype(np.uint8)

In [ ]:
plt.imshow()

In [ ]:
dist_transform = cv2.distanceTransform(img_,cv2.DIST_L2,5)

In [ ]:
plt.imshow(dist_transform)

In [ ]:
# noise removal
kernel = np.ones((3,3),np.uint8)
opening = cv2.morphologyEx(thresh,cv2.MORPH_OPEN,kernel, iterations = 2)

# sure background area
sure_bg = cv2.dilate(opening,kernel,iterations=3)

ret, sure_fg = cv2.threshold(dist_transform,0.3*dist_transform.max(),255,0)
plt.imshow(sure_fg)

In [ ]:
print(img_.dtype)
img_ = np.tile(img_, (1, 1, 3))
print(img_.shape)
cv2.watershed(img_, markers)

In [ ]:
markers = markers+1

# Now, mark the region of unknown with zero
markers[unknown==255] = 0

In [ ]:
img_t.dtype

In [ ]:
markers = cv2.watershed(img_t,markers)
img[markers == -1] = [255,0,0]